In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import visualization_utils as vis_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction, Box, Rgba, WeldJoint, GeometrySet
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from scipy.spatial.transform import Rotation as sp_rot
from sampling_based_motion_planners import StraightLineCollisionChecker, PRM, BiRRT


from pydrake.all import (RationalForwardKinematics, ModelInstanceIndex,
                         SpatialInertia, RevoluteJoint, FixedOffsetFrame)
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [ ]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [ ]:
meshcat = StartMeshcat()
meshcat.Delete()

In [ ]:
meshcat.Delete()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)

link_dimensions = (0.1, 0.2, 0.001)
pendulum_box = Box(*link_dimensions)
pendulum_inertia = SpatialInertia.SolidBoxWithDensity(1, *link_dimensions)
pendulum_model_instance = plant.AddModelInstance("pendulum")
def AddPendulumLink(parent_frame, link_index: int, color = np.array([1,0,0,1])):
    link_name = f"link_{link_index}"
    pend_body = plant.AddRigidBody(link_name, pendulum_model_instance, pendulum_inertia)
    parent_to_child_attach_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,link_dimensions[1]/2, 0)))
    )
    next_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,-link_dimensions[1]/2, 0)))
    )
    # add the revolute joint,
    joint_lim = np.pi
    plant.AddJoint(RevoluteJoint(
        f"joint_{link_index}", parent_frame, parent_to_child_attach_frame, [0,0,1],
        -joint_lim, joint_lim,
    )
    )
    
    plant.RegisterCollisionGeometry(pend_body, RigidTransform(),
                                    pendulum_box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(pend_body, RigidTransform(),
                                pendulum_box, link_name, color)
    
    
    
    return next_frame

def N_Link_Pendulum(n: int):
    assert n > 1
    parent_frame = plant.world_frame()
    colors = np.array(vis_utils.n_colors(n))/255
    colors = np.hstack([colors,np.ones((colors.shape[0],1))])
    for i in range(n):
        parent_frame = AddPendulumLink(parent_frame, i+1, colors[i])
    

        
obstacle_model_instance = plant.AddModelInstance("obstacle")
def AddRandomBox(index: int, pos_limits, size_limits, color = [0,0,0,1]):
    link_name = f"obstacle_{index}"     
    
    center = np.append(np.random.uniform(pos_limits[0],pos_limits[1], 2),0)
    
    random_rp = np.zeros(3)
    lim = np.pi/10
    random_rp[2] = np.random.uniform(-lim, lim)
    angle = RotationMatrix(RollPitchYaw(random_rp))
    
    pose = RigidTransform(p=center)@RigidTransform(R=angle)
    
    dimensions = np.append(np.random.uniform(size_limits[0],size_limits[1],2),0.001)
    box = Box(*dimensions)
    inertia = SpatialInertia.SolidBoxWithDensity(1, *dimensions)
    
    body = plant.AddRigidBody(link_name, obstacle_model_instance, inertia)
    plant.AddJoint(WeldJoint(link_name, plant.world_frame(), body.body_frame(),
                            pose))
    plant.RegisterCollisionGeometry(body, RigidTransform(),
                                    box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(body, RigidTransform(),
                                box, link_name, color)
    
    

def AddNRandomBoxes(n: int, pos_limits=(-10,10),
                    size_limits = (0.1,1), color= [0,0,0,1],
                   filter_all = False):
    assert n > 1
    for i in range(n):
        AddRandomBox(i, pos_limits, size_limits, color)

# child_frame = AddPendulumLink(plant.world_frame(), 1)
num_pend = 3
N_Link_Pendulum(num_pend)

num_obstacles = 5
obstacle_pos_limits = link_dimensions[1]*num_pend*np.array([-1,1])
obstacle_size_limits = (0.01,0.5)
AddNRandomBoxes(num_obstacles, obstacle_pos_limits, obstacle_size_limits)

plant.Finalize()

end_effector = plant.GetBodyByName(f"link_{num_pend}")

def SetMeshcatPerspective2d(meshcat_instance, X_WC):
    meshcat_instance.SetTransform("/Cameras/default", X_WC)
    meshcat_instance.SetProperty("/Background", "visible", False);
    meshcat_instance.SetProperty("/Grid", "visible", False);
    meshcat_instance.SetProperty("/Axes", "visible", False)

SetMeshcatPerspective2d(meshcat, RigidTransform(p=np.array([0,0,1])))
# meshcat.Set2dRenderMode(RigidTransform(p=np.array([0,0,1])))

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

q_star = np.zeros(plant.num_positions())
Ratfk = RationalForwardKinematics(plant)

diagram_vis_context = diagram.CreateDefaultContext()
plant_vis_context =  diagram.GetMutableSubsystemContext(plant, diagram_vis_context)
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_vis_context, plant, plant_vis_context,
    Ratfk, meshcat, q_star
)

In [ ]:
#compute limits in s-space
limits_s = []
lower_q = plant.GetPositionLowerLimits()
upper_q = plant.GetPositionUpperLimits()


# lower_q[0] = -np.pi/2
# upper_q[0] = np.pi/2

# lower_q[-1] = 0
# upper_q[-1] = 0

# lower_q[2] = 0

for q in [lower_q,upper_q]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)
print(limits_s)

In [ ]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_col_context)
def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1
    
def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=plant.GetPositions(plant_context)[i],
                                       description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)

    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

In [ ]:
col_checker = StraightLineCollisionChecker(check_collision_s_by_query, 100)
def sample_col_free_point():
    q = np.random.uniform(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
    s = Ratfk.ComputeSValue(q, q_star)
    return s
np.random.seed(0)
prm = PRM(sample_col_free_point,
         100,
         col_checker,
         dist_thresh = 100)
prm.draw_tree(vis_bundle, end_effector)

In [ ]:
def make_line_polys(prm):
    polys = np.empty(shape=(plant.num_positions(), len(prm.prm.edges())), dtype = object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))
    return polys

path_safe = make_line_polys(prm)
# path_unsafe = make_line_polys(PRM_bad)
    

In [ ]:
# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 4)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")


cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [ ]:
t0 = time.time()
ret = cspace_free_path.FindSeparationCertificateGivenPath(path_safe, set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

In [ ]:

def plot_pair_safety(pairs, is_safe_vec, body,
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(f"{prefix}_safe")
    meshcat.Delete(f"{prefix}_unsafe")
    
    safe_options = vis_utils.TrajectoryVisualizationOptions(
        path_color = Rgba(0,1,0,1), path_size = path_size, num_points = num_points
    )
    unsafe_options = vis_utils.TrajectoryVisualizationOptions(
        path_color = Rgba(1,0,0,1), path_size = path_size, num_points = num_points
    )
    
    for idx, (s0, s1) in enumerate(pairs):
        s0, s1 = np.array(s0), np.array(s1)
        if is_safe_vec[idx]:
            vis_utils.visualize_s_space_segment(vis_bundle,s0, s1, body,
                                        f"{prefix}_safe/seg_{idx}", safe_options)
        else:
            vis_utils.visualize_s_space_segment(vis_bundle,s0, s1, body, 
                                        f"{prefix}_unsafe/seg_{idx}", unsafe_options)
plot_pair_safety(prm.prm.edges(), ret[0], end_effector)

In [ ]:
animate_traj_s(traj, 1000, 1)

In [ ]:
unsafe_traj.shape